In [1]:
from time import perf_counter

from qiskit_aer.primitives import Sampler

from qtpu.compiler.compiler import compile_reach_size
from qtpu.evaluate import evaluate_sampler

from benchmark.ckt import run_qtpu, cut_ckt, run_ckt
from benchmark.ansatz import generate_ansatz
from benchmark.util import append_to_csv

In [2]:
QPU_SIZES = [5, 20]
XS, S, M, L, XL = 15, 30, 70, 100, 200

In [3]:
benchmarks = [
    {"name": "vqe", "num_qubits": size, "depth": d}
    for size in [XS, S, M]
    for d in [3]
]

In [4]:
def compile_and_run_qtpu(circuit, qpu_size):
    start = perf_counter()
    circuit = circuit.copy()
    circuit.measure_all()
    cut_circ = compile_reach_size(circuit, qpu_size, show_progress_bar=True)
    # print(cut_circ)
    compile_time = perf_counter() - start
    # evaluate_sampler(Sampler(), 10000)
    res, data = run_qtpu(cut_circ, .5)
    print(res)

    return {"qtpu_compile": compile_time, **data}


def compile_and_run_ckt(circuit, qpu_size):
    start = perf_counter()
    cut_circ = cut_ckt(circuit, qpu_size)
    compile_time = perf_counter() - start

    res, data = run_ckt(cut_circ, Sampler(), 100)
    print(res)

    return {"ckt_compile": compile_time, **data}

In [5]:
QPU_SIZE = 5
CSV = "runtime_results2.csv"

for bench in benchmarks:
    circuit = generate_ansatz(**bench)
    qtpu_res = compile_and_run_qtpu(circuit, QPU_SIZE)
    print(qtpu_res)
    ckt_res = compile_and_run_ckt(circuit, QPU_SIZE)
    print(ckt_res)
    append_to_csv(CSV, {**bench, **qtpu_res})

  0%|          | 0/100 [00:00<?, ?it/s]

[FrozenTrial(number=0, state=1, values=[186840.0, 0.9955098858248878], datetime_start=datetime.datetime(2024, 8, 10, 21, 38, 45, 44681), datetime_complete=datetime.datetime(2024, 8, 10, 21, 38, 45, 53534), params={'compress': 'qubits', 'choose_leaf': 'qubits', 'random_strength': 3.911296561246107, 'weight_edges': 'const', 'imbalance': 0.042493818406859535, 'imbalance_decay': 4.981026482690801, 'parts': 6, 'parts_decay': 0.6609854898472375, 'mode': 'recursive', 'objective': 'cut', 'fix_output_nodes': ''}, user_attrs={'ir': <qtpu.compiler.compress.CompressedIR object at 0x7f61ee711d10>, 'tree': <ContractionTree(N=15, branches=5, complete=False)>}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, distributions={'compress': CategoricalDistribution(choices=('qubits', '2q', 'none')), 'choose_leaf': CategoricalDistribution(choices=('qubits',)), 'random_strength': FloatDistribution(high=10.0, log=False, low=0.01, step=None), 'weight_edges': CategoricalDistribution(choices=('const'